# A widget for interactive analysis of a matchup

## Input parameters.

In [128]:
TEAM_1 = 'beastcoast'
TEAM_2 = 'j.storm'

FORCE_REFIT = False

LOGISTIC_SCALE = 3.0
COV_FUNC_NAME = 'exponential'
COV_FUNC_KWARGS = {'scale': 1.25 * 365 * 24 * 60 * 60 * 1000}
RADI_PRIOR_SD = 3.0

In [129]:
TEAM_1 = TEAM_1.lower()
TEAM_2 = TEAM_2.lower()

## Load libraries.

In [1]:
import os
while 'src' not in os.listdir():
    os.chdir('..')
os.getcwd()

'/Users/yl3/github/dota2'

In [36]:
import dill
import pandas as pd

import src.load
import src.models.gp
import src.stats

## Load data.

In [110]:
def load_premium_prof_matches():
    prem_matches = src.load.fetch_matches()
    prof_matches = src.load.fetch_matches('professional')
    matches = src.load.MatchDF(pd.concat([prem_matches.df, prof_matches.df]))
    return prem_matches

In [111]:
def _match_date(matches):
    """Compute the match dates of each team by team name.

    Args:
        matches (load.MatchDF): The match data.
    """
    radi_teams = matches.df[['radiant_name', 'radiant_valveId',
                             'radiant_nicknames', 'radiant_players',
                             'startDate']]
    dire_teams = matches.df[['dire_name', 'dire_valveId',
                             'dire_nicknames', 'dire_players',
                             'startDate']]
    colnames = ['team_name', 'team_id', 'player_names', 'player_ids',
                'startDate']
    radi_teams.columns = colnames
    dire_teams.columns = colnames
    teams_by_date = (pd.concat([radi_teams, dire_teams])
                     .sort_values('startDate')
                     .reset_index())
    for colname in ('matchId', 'team_id'):
        teams_by_date[colname] = \
            teams_by_date[colname].astype(pd.Int64Dtype())
    teams_by_date['team_name'] = teams_by_date['team_name'].str.lower()
    teams_by_date.index = teams_by_date['team_name']
    return teams_by_date

In [112]:
serialised_model_file = 'interactive/latest_fitted_gp_model.dill'
if FORCE_REFIT or not os.path.isfile(serialised_model_file):
    matches = load_premium_prof_matches()
    gp_model = src.models.gp.SkillsGPMAP.from_match_df(
        matches,
        cov_func_name=COV_FUNC_NAME,
        cov_func_kwargs=COV_FUNC_KWARGS,
        radi_prior_sd=RADI_PRIOR_SD,
        logistic_scale=LOGISTIC_SCALE
    )
    gp_model.fit()
    display(gp_model.fitted)
    match_pred = src.stats.MatchPred(
    matches,
    gp_model.fitted_pred_df(),
    gp_model.logistic_scale,
    gp_model.fitted_skills_mat())
    with open(serialised_model_file, 'wb') as fh:
        dill.dump(match_pred, fh)
else:
    with open(serialised_model_file, 'rb') as fh:
        match_pred = dill.load(fh)

### Compute team selectors.

In [130]:
loc1 = ((match_pred.df.radiant_name.str.lower() == TEAM_1)
        | (match_pred.df.dire_name.str.lower() == TEAM_1))
loc2 = ((match_pred.df.radiant_name.str.lower() == TEAM_2)
        | (match_pred.df.dire_name.str.lower() == TEAM_2))
if sum(loc1):
    team1_id = \
        match_pred.teams.index[match_pred.teams.str.lower() == TEAM_1][-1]
if sum(loc2):
    team2_id = \
        match_pred.teams.index[match_pred.teams.str.lower() == TEAM_2][-1]

# Get the most recent roster of both teams.
match_dates = _match_date(match_pred)
team_1_players = match_dates.loc[
    match_dates.team_id == team1_id, 'player_ids'].iloc[-1]
team_2_players = match_dates.loc[
    match_dates.team_id == team2_id, 'player_ids'].iloc[-1]
    
display(match_pred.teams[match_pred.teams.str.lower() == TEAM_1])
display(match_pred.teams[match_pred.teams.str.lower() == TEAM_2])
print(f"{sum(loc1)} matches for team 1.")
print(f"{sum(loc2)} matches for team 2.")

7079109    beastcoast
dtype: object

6288801    J.Storm
7409177    J.Storm
dtype: object

55 matches for team 1.
191 matches for team 2.


## Actual interactive plots

### Previous matches of both teams.

In [131]:
match_pred.df.loc[loc1].tail(20)

,startDate,league_name,radiant_name,dire_name,radiantVictory,radiant_nicknames,dire_nicknames,seriesId,duration,radiant_valveId,...,dire_players,league_id,startTimestamp,series_start_time,match_i_in_series,pred_win_prob,radi_skill,dire_skill,pred_win_prob_unknown_side,radi_adv
matchId,,,,,,,,,,,,,,,,,,,,,
4901742097,2019-07-13 18:04:37,The International 2019,beastcoast,J.Storm,True,"(Brax, ixmike88, Ryoya, M-GOD, MoOz)","(Moo, Resolut1on, Fear, 1437, nine)",347697,2490,7079109,...,"(84429681, 86725175, 87177591, 87196890, 94786...",10749,1563041077000,1563041077000,0,0.505886,0.632571,0.889867,0.478636,0.327934
4901835876,2019-07-13 19:13:43,The International 2019,beastcoast,J.Storm,True,"(Brax, ixmike88, Ryoya, M-GOD, MoOz)","(Moo, Resolut1on, Fear, 1437, nine)",347697,2229,7079109,...,"(84429681, 86725175, 87177591, 87196890, 94786...",10749,1563045223000,1563041077000,1,0.505910,0.632373,0.889377,0.478660,0.327934
4901925673,2019-07-13 20:37:17,The International 2019,beastcoast,Newbee,False,"(Brax, ixmike88, Ryoya, M-GOD, MoOz)","(pieliedie, Sneyking, MSS, YawaR, CCnC)",347748,1847,7079109,...,"(6922000, 10366616, 86726887, 108452107, 22166...",10749,1563050237000,1563050237000,0,0.273441,0.631922,3.891553,0.252827,0.327934
4901998619,2019-07-13 21:37:01,The International 2019,beastcoast,Newbee,False,"(Brax, ixmike88, Ryoya, M-GOD, MoOz)","(pieliedie, Sneyking, MSS, YawaR, CCnC)",347748,1386,7079109,...,"(6922000, 10366616, 86726887, 108452107, 22166...",10749,1563053821000,1563050237000,1,0.273441,0.631684,3.891323,0.252827,0.327934
4903974414,2019-07-14 19:02:07,The International 2019,beastcoast,J.Storm,False,"(Brax, ixmike88, Ryoya, M-GOD, MoOz)","(Moo, Resolut1on, Fear, 1437, nine)",348080,2495,7079109,...,"(84429681, 86725175, 87177591, 87196890, 94786...",10749,1563130927000,1563130927000,0,0.506302,0.628344,0.880650,0.479049,0.327934
4904062522,2019-07-14 20:18:11,The International 2019,beastcoast,J.Storm,False,"(Brax, ixmike88, Ryoya, M-GOD, MoOz)","(Moo, Resolut1on, Fear, 1437, nine)",348080,1817,7079109,...,"(84429681, 86725175, 87177591, 87196890, 94786...",10749,1563135491000,1563130927000,1,0.506359,0.628342,0.879962,0.479106,0.327934
4925450824,2019-07-25 19:00:01,DOTA Summit 10,beastcoast,J.Storm,False,"(Brax, ixmike88, Ryoya, M-GOD, MoOz)","(Moo, Resolut1on, Fear, 1437, nine)",351576,2952,7079109,...,"(84429681, 86725175, 87177591, 87196890, 94786...",11162,1564081201000,1564081201000,0,0.524450,0.668562,0.702859,0.497151,0.327934
4925548100,2019-07-25 20:20:13,DOTA Summit 10,J.Storm,beastcoast,False,"(Moo, Resolut1on, Fear, 1437, nine)","(Brax, ixmike88, Ryoya, M-GOD, MoOz)",351576,2793,6288801,...,"(31818853, 86715129, 115141430, 131706718, 349...",11162,1564086013000,1564081201000,1,0.530037,0.701924,0.668983,0.502737,0.327934
4925641710,2019-07-25 21:56:00,DOTA Summit 10,compLexity Gaming,beastcoast,False,"(Limmp, Zfreek, Meracle, 343, tavo)","(Brax, ixmike88, Ryoya, M-GOD, MoOz)",351615,2476,3,...,"(31818853, 86715129, 115141430, 131706718, 349...",11162,1564091760000,1564091760000,0,0.469511,-0.025033,0.669226,0.442571,0.327934


In [121]:
match_pred.df.loc[loc2].tail(20)

,startDate,league_name,radiant_name,dire_name,radiantVictory,radiant_nicknames,dire_nicknames,seriesId,duration,radiant_valveId,...,dire_players,league_id,startTimestamp,series_start_time,match_i_in_series,pred_win_prob,radi_skill,dire_skill,pred_win_prob_unknown_side,radi_adv
matchId,,,,,,,,,,,,,,,,,,,,,
4904171017,2019-07-14 22:09:50,The International 2019,Newbee,J.Storm,False,"(pieliedie, Sneyking, MSS, YawaR, CCnC)","(Moo, Resolut1on, Fear, 1437, nine)",348164,2034,6214538,...,"(84429681, 86725175, 87177591, 87196890, 94786...",10749,1563142190000,1563142190000,0,0.752309,3.883627,0.878665,0.730841,0.327934
4904218149,2019-07-14 23:14:58,The International 2019,J.Storm,Newbee,False,"(Moo, Resolut1on, Fear, 1437, nine)","(pieliedie, Sneyking, MSS, YawaR, CCnC)",348164,1956,6288801,...,"(6922000, 10366616, 86726887, 108452107, 22166...",10749,1563146098000,1563142190000,1,0.290564,0.877658,3.883537,0.269099,0.327934
4904261164,2019-07-15 00:19:24,The International 2019,Newbee,J.Storm,True,"(pieliedie, Sneyking, MSS, YawaR, CCnC)","(Moo, Resolut1on, Fear, 1437, nine)",348164,1901,6214538,...,"(84429681, 86725175, 87177591, 87196890, 94786...",10749,1563149964000,1563142190000,2,0.752410,3.883352,0.876758,0.730948,0.327934
4904311356,2019-07-15 01:30:53,The International 2019,J.Storm,Newbee,False,"(Moo, Resolut1on, Fear, 1437, nine)","(pieliedie, Sneyking, MSS, YawaR, CCnC)",348164,1964,6288801,...,"(6922000, 10366616, 86726887, 108452107, 22166...",10749,1563154253000,1563142190000,3,0.290472,0.875850,3.883057,0.269012,0.327934
4925176841,2019-07-25 16:05:44,DOTA Summit 10,Alliance,J.Storm,True,"(iNSaNiA, Boxi, qojqva, miCKe, Taiga)","(Moo, Resolut1on, Fear, 1437, nine)",351515,2709,111474,...,"(84429681, 86725175, 87177591, 87196890, 94786...",11162,1564070744000,1564070744000,0,0.780065,4.174722,0.704514,0.760173,0.327934
4925317287,2019-07-25 17:28:34,DOTA Summit 10,Alliance,J.Storm,True,"(iNSaNiA, Boxi, qojqva, miCKe, Taiga)","(Moo, Resolut1on, Fear, 1437, nine)",351515,2143,111474,...,"(84429681, 86725175, 87177591, 87196890, 94786...",11162,1564075714000,1564070744000,1,0.780118,4.174806,0.703678,0.760228,0.327934
4925450824,2019-07-25 19:00:01,DOTA Summit 10,beastcoast,J.Storm,False,"(Brax, ixmike88, Ryoya, M-GOD, MoOz)","(Moo, Resolut1on, Fear, 1437, nine)",351576,2952,7079109,...,"(84429681, 86725175, 87177591, 87196890, 94786...",11162,1564081201000,1564081201000,0,0.524450,0.668562,0.702859,0.497151,0.327934
4925548100,2019-07-25 20:20:13,DOTA Summit 10,J.Storm,beastcoast,False,"(Moo, Resolut1on, Fear, 1437, nine)","(Brax, ixmike88, Ryoya, M-GOD, MoOz)",351576,2793,6288801,...,"(31818853, 86715129, 115141430, 131706718, 349...",11162,1564086013000,1564081201000,1,0.530037,0.701924,0.668983,0.502737,0.327934
4925748555,2019-07-26 00:17:23,DOTA Summit 10,paiN Gaming,J.Storm,True,"(Stormwind, LESLÃO, hFn, Thiolicor, Unknown)","(Moo, Resolut1on, Fear, 1437, nine)",351637,2355,67,...,"(84429681, 86725175, 87177591, 87196890, 94786...",11162,1564100243000,1564100243000,0,0.690702,2.782078,0.699804,0.666434,0.327934


### Team skills

In [132]:
match_pred.team_skills_plot(
    match_pred.teams.index[
        match_pred.teams.str.lower().isin([TEAM_1, TEAM_2])
    ])

### Player skills

In [115]:
match_pred.player_skills_plot(team_1_players + team_2_players)